<a href="https://colab.research.google.com/github/vinayR-cmd/Aadhaar_risk/blob/main/UIADI_ML_WEB_APP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# monthly_df_bio['month_name'] = monthly_df_bio['month'].dt.to_timestamp().dt.month_name()

In [ ]:
!pip install nbstripout
!nbstripout --keep-output UIADI_ML_WEB_APP.ipynb


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df1=pd.read_csv('/content/monthly_enrolment.csv')
df2=pd.read_csv('/content/monthly_biometric.csv')

In [ ]:
df1.shape, df2.shape

In [ ]:
df1.columns, df2.columns

In [ ]:
df1['month_name'] = pd.to_datetime(df1['month']).dt.month_name()

In [ ]:
df1.shape, df2.shape

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df1.columns, df2.columns

In [ ]:
df1.info(), df2.info()

In [ ]:
# Ensure month is Period[M] in both
df1['month'] = df1['month'].astype('period[M]')
df2['month'] = df2['month'].astype('period[M]')


In [ ]:
df1.info(), df2.info()

In [ ]:
merged_df = pd.merge(
    df1,
    df2,
    on=['state', 'district', 'month'],
    how='inner',
    suffixes=('_enrol', '_bio')
)

merged_df.shape


In [ ]:
merged_df.head()

In [ ]:
merged_df.info()

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(2,1,1)
sns.kdeplot(x='risk_score', data=merged_df, fill=True)
plt.xlabel('kde plot of enrollment risk score')

plt.subplot(2,1,2)
sns.kdeplot(x='bio_risk_score', data=merged_df, fill=True)
plt.xlabel('kde plot of biometric risk score')
plt.show()

In [ ]:
merged_df = merged_df.sort_values(['state', 'district', 'month'])
merged_df.head()


In [ ]:
df2['month'].unique()

In [ ]:
df1['month'].unique()

In [ ]:
merged_df['month'].unique()

In [ ]:
merged_df['month_name_bio'].unique()

In [ ]:
merged_df=merged_df.rename(columns={
    'month_name':'month_name_enrol',
    'risk_score':'enrollment_risk_score',

})

In [ ]:
merged_df.columns

In [ ]:
len(merged_df['state'].unique())

In [ ]:
merged_df['district'].nunique()

In [ ]:
merged_df.isnull().sum()

In [ ]:
merged_df.duplicated().sum()
#

In [ ]:
# univariate analysis

In [ ]:
num_cols=merged_df.select_dtypes(include=['number']).columns.tolist()
cat_cols=merged_df.select_dtypes(include=['object','category']).columns.tolist()

In [ ]:
num_cols, cat_cols

In [ ]:
plt.figure()
plt.scatter(
    merged_df['enrollment_risk_score'],
    merged_df['bio_risk_score'],
    alpha=0.4
)
plt.xlabel("Enrolment Risk Score")
plt.ylabel("Biometric Update Risk Score")
plt.title("Enrolment Risk vs Biometric Update Risk")
plt.show()


In [ ]:
sns.scatterplot(x='enrollment_risk_score',y='bio_risk_score', hue='month_name_bio', data=merged_df)

In [ ]:
merged_df.groupby('month')[['enrollment_risk_score','bio_risk_score']].mean().plot()
plt.title("Monthly Trend: Enrolment Risk vs Biometric Risk")
plt.ylabel("Average Risk Score")
plt.show()


In [ ]:
sns.scatterplot(x='total_enrollments',y='bio_risk_score', data=merged_df)

In [ ]:
sns.scatterplot(x='child_enrolment_ratio',y='child_bio_ratio', data=merged_df)

In [ ]:
district_volatility = (
    merged_df.groupby(['state','district'])['bio_risk_score']
    .std()
    .sort_values(ascending=False)
    .head(10)
)

district_volatility.plot(kind='bar')
plt.title("Most Volatile Districts (Biometric Risk)")
plt.ylabel("Risk Variability")
plt.show()

# High volatility districts require continuous monitoring rather than one-time interventions.


In [ ]:
merged_df.head()

In [ ]:
len(num_cols), cat_cols

In [ ]:
plt.figure(figsize=(25,10))

for i ,col in enumerate(num_cols, 1):
  plt.subplot(3,5,i)
  sns.kdeplot(merged_df[col], fill=True)
  plt.xlabel(f'kde plot {col}')

In [ ]:
plt.figure(figsize=(30,10))

for i ,col in enumerate(num_cols, 1):
  plt.subplot(3,5,i)
  sns.boxplot(merged_df[col])
  plt.xlabel(f'boc plot of {col}')

In [ ]:
merged_df.head()

In [ ]:
# month wise total_enrollments ansd total_bio_updates
plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
sns.barplot(x='month_name_enrol',y='total_enrollments', data=merged_df)
plt.xticks(rotation=90)

plt.subplot(2,1,2)
sns.barplot(x='month_name_bio',y='total_bio_updates', data=merged_df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
!pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport
profile=ProfileReport(merged_df, title='Pandas Profiling Report', explorative=True)
profile.to_file("merged_eda.html")
profile

In [ ]:
merged_df.head()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(12,8))

# Child ratios
child_ratios = (
    merged_df.groupby('month')[
        ['child_enrolment_ratio', 'child_bio_ratio']
    ].mean()
)

child_ratios.plot(
    ax=axes[0],
    title='Enrollment vs Biometric Ratio Trends (Child)'
)
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Ratio')
axes[0].grid(True)

# Adult ratios
adult_ratios = (
    merged_df.groupby('month')[
        ['adult_enrolment_ratio', 'adult_bio_ratio']
    ].mean()
)

adult_ratios.plot(
    ax=axes[1],
    title='Enrollment vs Biometric Ratio Trends (Adult)'
)
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Ratio')
axes[1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(12,8))

# Child ratios
child_ratios = (
    merged_df.groupby('state')[
        ['child_enrolment_ratio', 'child_bio_ratio']
    ].mean()
)

child_ratios.plot(
    ax=axes[0],
    title='Enrollment vs Biometric Ratio Trends (Child)'
)
axes[0].set_xlabel('state')
axes[0].set_ylabel('Ratio')
axes[0].grid(True)

# Adult ratios
adult_ratios = (
    merged_df.groupby('state')[
        ['adult_enrolment_ratio', 'adult_bio_ratio']
    ].mean()
)

adult_ratios.plot(
    ax=axes[1],
    title='Enrollment vs Biometric Ratio Trends (Adult)'
)
axes[1].set_xlabel('state')
axes[1].set_ylabel('Ratio')
axes[1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14,10))

child_ratios = merged_df.groupby('state')[
    ['child_enrolment_ratio', 'child_bio_ratio']
].mean()

child_ratios.plot(ax=axes[0], title='Child Enrollment vs Biometric Ratio')
axes[0].set_xticks(range(len(child_ratios.index)))
axes[0].set_xticklabels(child_ratios.index, rotation=90)
axes[0].grid(True)

adult_ratios = merged_df.groupby('state')[
    ['adult_enrolment_ratio', 'adult_bio_ratio']
].mean()

adult_ratios.plot(ax=axes[1], title='Adult Enrollment vs Biometric Ratio')
axes[1].set_xticks(range(len(adult_ratios.index)))
axes[1].set_xticklabels(adult_ratios.index, rotation=90)
axes[1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

risk_ratio = merged_df.groupby('state')[
    ['enrollment_risk_score', 'bio_risk_score']
].mean()

fig, ax = plt.subplots(figsize=(14,6))

risk_ratio.plot(ax=ax)

ax.set_title('State-wise Enrollment Risk vs Biometric Risk')
ax.set_xlabel('State')
ax.set_ylabel('Risk Score')

# force all state labels to show
ax.set_xticks(range(len(risk_ratio.index)))
ax.set_xticklabels(risk_ratio.index, rotation=90)

ax.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
num_cols, cat_cols

In [ ]:
plt.figure(figsize=(7,7))
sns.scatterplot(x='bio_risk_score',y='enrollment_risk_score', data=merged_df, hue='month_name_bio')

In [ ]:
sns.jointplot(
    x='bio_risk_score',
    y='enrollment_risk_score',
    data=merged_df,
    kind='reg',
    scatter_kws={'alpha':0.4, 's':30},
    line_kws={'color':'red', 'linewidth':3}
)
plt.show()


In [ ]:
merged_df.head()

In [ ]:
sns.boxplot(x='month_name_bio',y='bio_risk_score', data=merged_df)
plt.xticks(rotation=90)
#

In [ ]:
sns.boxplot(x='month_name_bio',y='enrollment_risk_score', data=merged_df)
plt.xticks(rotation=90)
#

In [ ]:
sns.heatmap(merged_df[num_cols].corr(), annot=True,fmt='.1f', cmap='coolwarm', linewidth=0.6)
#

In [ ]:
import plotly.express as px
# age_0_5	age_5_17	age_18_greater
fig = px.scatter_3d(
    merged_df,
    x='age_0_5',
    y='age_5_17',
    z='age_18_greater',
    color='enrollment_risk_score',
    opacity=0.7,
    title='3D Scatter: child vsTeen vs Adult Enrollment'
)

fig.update_layout(
    scene=dict(
        xaxis_title='child Enrollment Ratio',
        yaxis_title='teen Enrollment Ratio',
        zaxis_title='adult Enrollment Ratio'
    )
)

fig.show()


# Task
Visualize the relationship between 'total_enrollments' and 'total_bio_updates' across districts or states using a scatter plot to see if higher enrollment numbers correlate with higher biometric update activities.

## Correlation Analysis of Key Metrics

### Subtask:
Calculate and visualize the correlation matrix for key numerical columns, specifically focusing on the relationships between enrollment and biometric ratios and risk scores.


**Reasoning**:
First, I will select the relevant numerical columns from the `merged_df` DataFrame as specified in the instructions for correlation analysis.



In [ ]:
selected_cols = [
    'child_enrolment_ratio',
    'teen_enrolment_ratio',
    'adult_enrolment_ratio',
    'enrollment_risk_score',
    'child_bio_ratio',
    'adult_bio_ratio',
    'bio_risk_score'
]

correlation_matrix = merged_df[selected_cols].corr()
correlation_matrix.head()

**Reasoning**:
Now that the correlation matrix has been calculated, I will visualize it using a heatmap to clearly display the relationships between the selected numerical columns, as per the instructions.



In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
plt.title('Correlation Matrix of Enrollment and Biometric Ratios/Risk Scores')
plt.show()

## Relationship between Total Enrollments and Biometric Updates

### Subtask:
Create a scatter plot to visualize the relationship between 'total_enrollments' and 'total_bio_updates' across states to see if higher enrollment numbers correlate with higher biometric update activities.


**Reasoning**:
First, I will group the `merged_df` DataFrame by 'state' and calculate the sum of 'total_enrollments' and 'total_bio_updates' to prepare the data for the scatter plot, as instructed.



In [ ]:
state_summary = merged_df.groupby('state')[['total_enrollments', 'total_bio_updates']].sum().reset_index()
state_summary.head()

**Reasoning**:
Now that the `state_summary` DataFrame is prepared, I will create a scatter plot to visualize the relationship between 'total_enrollments' and 'total_bio_updates' across states, as specified in the instructions.



In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='total_enrollments',
    y='total_bio_updates',
    data=state_summary,
    s=100, # Adjust marker size for better visibility
    alpha=0.7 # Adjust transparency
)
plt.xlabel('Total Enrollments')
plt.ylabel('Total Biometric Updates')
plt.title('Total Enrollments vs. Total Biometric Updates by State')
plt.grid(True)
plt.show()

## Analyze State-wise Ratio Discrepancies

### Subtask:
Calculate the difference between 'child_enrolment_ratio' and 'child_bio_ratio', and 'adult_enrolment_ratio' and 'adult_bio_ratio' for each state. Then, visualize these differences to highlight states where there are significant gaps or alignments between enrollment and biometric update activities for different age groups.


**Reasoning**:
First, I will group the `merged_df` DataFrame by 'state' and calculate the mean of the specified ratio columns (`child_enrolment_ratio`, `child_bio_ratio`, `adult_enrolment_ratio`, and `adult_bio_ratio`). This step prepares the data for calculating the differences and subsequent visualization, as per the instructions.



In [ ]:
state_ratios = merged_df.groupby('state')[['child_enrolment_ratio', 'child_bio_ratio', 'adult_enrolment_ratio', 'adult_bio_ratio']].mean().reset_index()
state_ratios.head()

**Reasoning**:
Now that the mean ratios for each state have been calculated, I will proceed to calculate the differences between enrollment and biometric ratios for both children and adults. Then, I will visualize these differences using bar plots to highlight discrepancies across states, as specified in the instructions.



In [ ]:
state_ratios['child_ratio_diff'] = state_ratios['child_enrolment_ratio'] - state_ratios['child_bio_ratio']
state_ratios['adult_ratio_diff'] = state_ratios['adult_enrolment_ratio'] - state_ratios['adult_bio_ratio']

fig, axes = plt.subplots(2, 1, figsize=(14, 12))

sns.barplot(x='state', y='child_ratio_diff', data=state_ratios.sort_values('child_ratio_diff', ascending=False), ax=axes[0], palette='viridis')
axes[0].set_title('Difference: Child Enrollment Ratio vs. Child Biometric Ratio by State')
axes[0].set_xlabel('State')
axes[0].set_ylabel('Child Ratio Difference (Enrollment - Biometric)')
axes[0].tick_params(axis='x', rotation=90)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

sns.barplot(x='state', y='adult_ratio_diff', data=state_ratios.sort_values('adult_ratio_diff', ascending=False), ax=axes[1], palette='magma')
axes[1].set_title('Difference: Adult Enrollment Ratio vs. Adult Biometric Ratio by State')
axes[1].set_xlabel('State')
axes[1].set_ylabel('Adult Ratio Difference (Enrollment - Biometric)')
axes[1].tick_params(axis='x', rotation=90)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

**Reasoning**:
The previous code generated a `FutureWarning` because `palette` was used without `hue` in `sns.barplot`. To resolve this, I will explicitly set `hue='state'` and `legend=False` in both `barplot` calls as suggested by the warning.



In [ ]:
state_ratios['child_ratio_diff'] = state_ratios['child_enrolment_ratio'] - state_ratios['child_bio_ratio']
state_ratios['adult_ratio_diff'] = state_ratios['adult_enrolment_ratio'] - state_ratios['adult_bio_ratio']

fig, axes = plt.subplots(2, 1, figsize=(14, 12))

sns.barplot(x='state', y='child_ratio_diff', data=state_ratios.sort_values('child_ratio_diff', ascending=False), ax=axes[0], palette='viridis', hue='state', legend=False)
axes[0].set_title('Difference: Child Enrollment Ratio vs. Child Biometric Ratio by State')
axes[0].set_xlabel('State')
axes[0].set_ylabel('Child Ratio Difference (Enrollment - Biometric)')
axes[0].tick_params(axis='x', rotation=90)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

sns.barplot(x='state', y='adult_ratio_diff', data=state_ratios.sort_values('adult_ratio_diff', ascending=False), ax=axes[1], palette='magma', hue='state', legend=False)
axes[1].set_title('Difference: Adult Enrollment Ratio vs. Adult Biometric Ratio by State')
axes[1].set_xlabel('State')
axes[1].set_ylabel('Adult Ratio Difference (Enrollment - Biometric)')
axes[1].tick_params(axis='x', rotation=90)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## Identify Top/Bottom States by Combined Risk

### Subtask:
Group the merged dataframe by 'state' and calculate the average 'enrollment_risk_score' and 'bio_risk_score'. Visualize the states with the highest and lowest average risk scores to identify potential areas requiring more attention or successful models.


**Reasoning**:
First, I will group the `merged_df` DataFrame by 'state' and calculate the mean of 'enrollment_risk_score' and 'bio_risk_score'. Then, I will create a new column 'combined_risk_score' by summing these two mean scores, as specified in the instructions.



In [ ]:
state_risk_scores = merged_df.groupby('state')[['enrollment_risk_score', 'bio_risk_score']].mean().reset_index()
state_risk_scores['combined_risk_score'] = state_risk_scores['enrollment_risk_score'] + state_risk_scores['bio_risk_score']
state_risk_scores.head()

**Reasoning**:
Now that the combined risk scores have been calculated for each state, I will identify the top 5 and bottom 5 states based on these scores, and store them in separate DataFrames to prepare for visualization.



In [ ]:
top_5_risk_states = state_risk_scores.sort_values(by='combined_risk_score', ascending=False).head(5)
bottom_5_risk_states = state_risk_scores.sort_values(by='combined_risk_score', ascending=True).head(5)

print('Top 5 states by combined risk score:')
print(top_5_risk_states)
print('\nBottom 5 states by combined risk score:')
print(bottom_5_risk_states)

**Reasoning**:
Now that the top and bottom 5 states by combined risk score have been identified, I will visualize their individual enrollment and biometric risk scores using bar plots to clearly show the differences and highlight areas needing attention or models for success.



In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# Plot for Top 5 States
top_5_risk_melted = top_5_risk_states.melt(
    id_vars='state',
    value_vars=['enrollment_risk_score', 'bio_risk_score'],
    var_name='Risk Type',
    value_name='Average Risk Score'
)
sns.barplot(x='state', y='Average Risk Score', hue='Risk Type', data=top_5_risk_melted, ax=axes[0], palette='viridis')
axes[0].set_title('Top 5 States by Combined Risk Score')
axes[0].set_xlabel('State')
axes[0].set_ylabel('Average Risk Score')
axes[0].tick_params(axis='x', rotation=90)
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

# Plot for Bottom 5 States
bottom_5_risk_melted = bottom_5_risk_states.melt(
    id_vars='state',
    value_vars=['enrollment_risk_score', 'bio_risk_score'],
    var_name='Risk Type',
    value_name='Average Risk Score'
)
sns.barplot(x='state', y='Average Risk Score', hue='Risk Type', data=bottom_5_risk_melted, ax=axes[1], palette='magma')
axes[1].set_title('Bottom 5 States by Combined Risk Score')
axes[1].set_xlabel('State')
axes[1].set_ylabel('Average Risk Score')
axes[1].tick_params(axis='x', rotation=90)
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## Monthly Trend of Risk Score Differences

### Subtask:
Calculate the monthly difference between 'enrollment_risk_score' and 'bio_risk_score' to observe how this discrepancy evolves over time. Plotting this trend can reveal periods where the gap between enrollment and biometric risks widens or narrows.


**Reasoning**:
First, I will group the `merged_df` DataFrame by 'month' and calculate the mean of 'enrollment_risk_score' and 'bio_risk_score' to prepare for calculating the monthly difference, as per the instructions.



In [ ]:
monthly_risk_trends = merged_df.groupby('month')[['enrollment_risk_score', 'bio_risk_score']].mean().reset_index()
monthly_risk_trends.head()

**Reasoning**:
Now that the `monthly_risk_trends` DataFrame contains the mean risk scores for each month, I will calculate the difference between 'enrollment_risk_score' and 'bio_risk_score' and store it in a new column called 'risk_score_difference'.



In [ ]:
monthly_risk_trends['risk_score_difference'] = monthly_risk_trends['enrollment_risk_score'] - monthly_risk_trends['bio_risk_score']
monthly_risk_trends.head()

**Reasoning**:
Now that the `risk_score_difference` has been calculated, I will create a line plot to visualize its monthly trend, as per the instructions, to observe how the discrepancy between enrollment and biometric risk scores evolves over time.



In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(monthly_risk_trends['month'].astype(str), monthly_risk_trends['risk_score_difference'], marker='o')
plt.title('Monthly Trend of Enrollment vs. Biometric Risk Score Difference')
plt.xlabel('Month')
plt.ylabel('Risk Score Difference')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

## Final Task

### Subtask:
Summarize the key patterns and insights discovered from the analysis, drawing conclusions about the relationships between enrollment and biometric update processes, and identifying areas of concern or success.


## Summary:

### Q&A
Higher enrollment numbers generally correlate with higher biometric update activities, as visually indicated by the scatter plot of 'total\_enrollments' versus 'total\_bio\_updates' across states. States with more enrollments also tend to have a greater volume of biometric updates.

### Data Analysis Key Findings
*   **Strong Positive Correlation between Child Enrollment and Risk**: A very strong positive correlation ($\sim$0.98) was observed between `child_enrolment_ratio` and `enrollment_risk_score`, suggesting that a higher child enrollment ratio is closely associated with an elevated enrollment risk.
*   **Perfect Negative Correlation in Biometric Ratios**: There is a perfect negative correlation (-1.00) between `child_bio_ratio` and `adult_bio_ratio`, indicating that as one increases, the other decreases proportionally. Similarly, `bio_risk_score` shows a perfect positive correlation (1.00) with `child_bio_ratio`, implying it is directly derived from it.
*   **Enrollment and Biometric Updates are Positively Related**: The scatter plot revealed a general positive relationship between total enrollments and total biometric updates by state. For example, 'Andhra Pradesh' had 125,277 total enrollments and 1,378,079 total biometric updates, while 'Andaman and Nicobar Islands' showed 511 total enrollments and 8,583 total biometric updates.
*   **Significant State-wise Discrepancies in Child and Adult Ratios**:
    *   Some states show large positive differences where enrollment ratios exceed biometric ratios (e.g., higher child enrollment than child bio-updates), while others show negative differences (more bio-updates than enrollments).
    *   The visualization highlighted specific states with substantial gaps, indicating potential areas of misalignment between enrollment and biometric update activities for both children and adults.
*   **Identification of High and Low-Risk States**: States like "Andaman and Nicobar Islands" were identified among the top 5 states with the highest combined risk scores (enrollment + biometric risk), while "Nagaland" was among the bottom 5 states with the lowest combined risk, indicating successful models or less concern.
*   **Biometric Risk Generally Higher Than Enrollment Risk**: The monthly trend analysis revealed that the `bio_risk_score` was generally higher than the `enrollment_risk_score`, resulting in negative risk score differences across observed months. For instance, in March 2025, the difference was approximately -0.156 (0.353 enrollment risk - 0.509 bio risk).

### Insights or Next Steps
*   Investigate the perfect correlations (e.g., between `child_bio_ratio` and `adult_bio_ratio`, and `bio_risk_score`) to understand the underlying data generation or calculation logic, as they might indicate derived metrics rather than independent variables.
*   Focus on states identified with significant discrepancies between enrollment and biometric update ratios (both child and adult) to understand the root causes of these gaps, which could range from policy implementation issues to data collection challenges.


In [ ]:
merged_df.info()

** yhs se ab hm log Project me ML wala start kr rhe**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
df_ml = merged_df.drop(
    columns=[
        'state',
        'district',
        'month_name_enrol',
        'month_name_bio'
    ]
).copy()


In [ ]:
# Convert Period[M] → timestamp → ordinal
df_ml['month_ts'] = df_ml['month'].dt.to_timestamp()
df_ml['month_num'] = df_ml['month_ts'].map(pd.Timestamp.toordinal)

df_ml.drop(columns=['month', 'month_ts'], inplace=True)


In [ ]:
y = df_ml['bio_risk_score']


In [ ]:
X = df_ml.drop(
    columns=[
        'bio_risk_score',
        'bio_age_5_17',
        'bio_age_17_',
        'total_bio_updates',
        'child_bio_ratio',
        'adult_bio_ratio'
    ]
)


In [ ]:
numeric_features = X.columns.tolist()
numeric_features


In [ ]:
numeric_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)


In [ ]:
# Sort by time before split
df_ml = df_ml.sort_values('month_num')

X = X.loc[df_ml.index]
y = y.loc[df_ml.index]

# 80% train, 20% test (time-based)
split_index = int(0.8 * len(df_ml))

X_train = X.iloc[:split_index]
X_test  = X.iloc[split_index:]

y_train = y.iloc[:split_index]
y_test  = y.iloc[split_index:]


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor


In [ ]:
ridge = Ridge()

ridge_param_grid = {
    'model__alpha': [0.01, 0.1, 1, 10, 100]
}


In [ ]:
rf = RandomForestRegressor(random_state=42)

rf_param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [None, 5, 10],
    'model__min_samples_split': [2, 5]
}


In [ ]:
pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', ridge)   # model will be tuned
    ]
)


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)


In [ ]:
ridge_grid = GridSearchCV(
    estimator=pipeline,
    param_grid=ridge_param_grid,
    cv=tscv,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

ridge_grid.fit(X_train, y_train)


In [ ]:
print("Best Ridge Params:", ridge_grid.best_params_)
print("Best Ridge MAE:", -ridge_grid.best_score_)


In [ ]:
pipeline_rf = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', rf)
    ]
)

rf_grid = GridSearchCV(
    estimator=pipeline_rf,
    param_grid=rf_param_grid,
    cv=tscv,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

rf_grid.fit(X_train, y_train)


In [ ]:
print("Best RF Params:", rf_grid.best_params_)
print("Best RF MAE:", -rf_grid.best_score_)


In [ ]:
# Select best Random Forest model
best_model = rf_grid.best_estimator_

print("Best model selected:")
print(best_model)



In [ ]:
import joblib
joblib.dump(best_model, "best_model.pkl")


In [ ]:
y_pred = best_model.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

print("Final MAE :", mean_absolute_error(y_test, y_pred))
print("Final RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("Final R2  :", r2_score(y_test, y_pred))


In [ ]:
import joblib

joblib.dump(best_model, "best_model.pkl")
print("Model saved as best_model.pkl")


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np

# -------------------------------
# App Config
# -------------------------------
st.set_page_config(
    page_title="Aadhaar Biometric Risk Analyzer",
    page_icon="🆔",
    layout="wide"
)

st.title("🆔 Aadhaar Biometric Risk Analyzer")
st.markdown("""
Predict **biometric update risk** using Aadhaar enrolment patterns
and generate **actionable governance insights**.
""")

# -------------------------------
# Load Model
# -------------------------------
@st.cache_resource
def load_model():
    return joblib.load("best_model.pkl")

model = load_model()

# -------------------------------
# Upload Data
# -------------------------------
st.header("📂 Upload Monthly Enrolment Data")
uploaded_file = st.file_uploader("Upload CSV file", type=["csv"])

if uploaded_file is None:
    st.info("Please upload a CSV file to proceed.")
    st.stop()

df = pd.read_csv(uploaded_file)

# -------------------------------
# Validate Columns
# -------------------------------
required_cols = [
    'state', 'district',
    'age_0_5', 'age_5_17', 'age_18_greater',
    'total_enrollments',
    'child_enrolment_ratio',
    'teen_enrolment_ratio',
    'adult_enrolment_ratio',
    'enrollment_risk_score',
    'month_num'
]

missing_cols = [c for c in required_cols if c not in df.columns]
if missing_cols:
    st.error(f"Missing required columns: {missing_cols}")
    st.stop()

# -------------------------------
# Prediction
# -------------------------------
X = df[[
    'age_0_5', 'age_5_17', 'age_18_greater',
    'total_enrollments',
    'child_enrolment_ratio',
    'teen_enrolment_ratio',
    'adult_enrolment_ratio',
    'enrollment_risk_score',
    'month_num'
]]

df['predicted_bio_risk'] = model.predict(X)

# -------------------------------
# Risk Categorization
# -------------------------------
def risk_level(x):
    if x <= 0.35:
        return "Low"
    elif x <= 0.50:
        return "Medium"
    else:
        return "High"

df['risk_level'] = df['predicted_bio_risk'].apply(risk_level)

# -------------------------------
# State-wise Summary
# -------------------------------
st.header("📊 State-wise Risk Summary")

state_summary = (
    df.groupby('state')['predicted_bio_risk']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

st.dataframe(state_summary)

st.metric("🔴 Highest Risk State", state_summary.iloc[0]['state'])
st.metric("🟢 Lowest Risk State", state_summary.iloc[-1]['state'])

st.subheader("State-wise Average Risk")
st.bar_chart(state_summary.set_index('state'))

# -------------------------------
# State Deep Dive
# -------------------------------
st.header("🏙️ State Deep Dive")

selected_state = st.selectbox("Select State", df['state'].unique())
state_df = df[df['state'] == selected_state]

district_summary = (
    state_df.groupby('district')['predicted_bio_risk']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

st.subheader("District-wise Risk Ranking")
st.dataframe(district_summary)
st.bar_chart(district_summary.set_index('district'))

# -------------------------------
# Recommendations
# -------------------------------
st.subheader("⚠️ Recommended Actions")

avg_state_risk = district_summary['predicted_bio_risk'].mean()

if avg_state_risk > 0.5:
    st.error("""
    🔴 High Risk:
    - Deploy biometric update camps
    - Enable OTP-based fallback authentication
    - Increase mobile enrolment units
    """)
elif avg_state_risk > 0.35:
    st.warning("""
    🟡 Medium Risk:
    - Monitor biometric trends
    - Schedule periodic update drives
    """)
else:
    st.success("""
    🟢 Low Risk:
    - Routine monitoring
    - No immediate action required
    """)

# -------------------------------
# Download Results
# -------------------------------
st.header("⬇️ Download Predictions")

csv = df.to_csv(index=False).encode("utf-8")
st.download_button(
    "Download Prediction CSV",
    csv,
    "biometric_risk_predictions.csv",
    "text/csv"
)


In [ ]:
!pip install streamlit pyngrok


In [ ]:
!ngrok authtoken 302sa1nncfXTexWdqZ2Gwul9mfF_2t1p5YRDNxSLqRzcHL5W9

In [ ]:
import os
import threading

def run_streamlit():
    os.system("streamlit run app.py --server.port 8501")

thread = threading.Thread(target=run_streamlit)
thread.start()


In [ ]:
from pyngrok import ngrok
import time

ngrok.kill()
time.sleep(3)

public_url = ngrok.connect(8501)
print("Your Streamlit app is live at:", public_url)


In [ ]:
import sklearn
import sys

print(sys.version)
print(sklearn.__version__)
